### Genauigkeit der Aussage berechnen

#### Beispiel von vorher

In [11]:
# FashionMNIST: https://github.com/zalandoresearch/fashion-mnist

import gzip
import numpy as np

def open_images(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1, 28, 28)\
            .astype(np.float32)


def open_labels(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=8)
    
X_train = open_images("data/fashion/train-images-idx3-ubyte.gz")
y_train_categories = open_labels("data/fashion/train-labels-idx1-ubyte.gz")

# Sequential: bedeutet, dass man Ebene für Ebene ein Modell definieren kann
from keras.models import Sequential
# Dense: Standard-Layer. Bedeutet, dass alle möglichen Pfeile zwischen allen Neuronen erstellt werden (alles mit allem verknüpfen)
from keras.layers import Dense

model = Sequential()

# Hidden layer 
#  100 Neuronen
#  input_shape: Eingabeformat der Daten (bsw. 784 Zahlen für das 28*28px Image)
#  (784,): Tuple - Python braucht für ein Tuple mit nur einem Wert ein ,
model.add(Dense(100, activation="sigmoid", input_shape=(784,)))

# Ausgabeschicht
#  Hier wird nur ein Neuron gebraucht
model.add(Dense(1, activation="sigmoid" ))

# generiert effizienteren Code. Nötig, falls es später auf einer GPU ausgeführt wird
# optimizer: Welches Verfahren soll verwendet werden, um die Gewichte zu verstellen (stochastic gradient descent)

c:\Users\vtewe\python_projects\udemy_ai\.conda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Beim Kompilieren kann man Metriken angeben, die mit ausgeben werden sollen, z.B. die Genauigkeit (accuracy)

In [12]:
# accuracy-Parameter übergeben 
model.compile(optimizer="sgd", loss="binary_crossentropy", metrics=["accuracy"])

# Daten ummappen, damit die Daten statt im Format "28x28" als 784 Zahlen representiert werden 
# 60000 Datensätze mit jeweils 784 Einträgen
X_train_mapped = X_train.reshape(60000, 784)

# Für die Ermittlung sollen nur T-Shirts (Kategorie 0) betrachtet werden. Deshalb wird das Array auf true/false (T-Shirt oder nicht) umgemappt
y_train = y_train_categories == 0

### Genauigkeit wird beim Trainieren mit ausgegeben
Nützlich, wenn das Trainieren länger läuft. Man hat so direktes Feedback, wie gut es bereits ist.

In [13]:
# epochs: wie oft die Daten komplett durchlaufen werden
# batch_size: schaue dir die ersten 1000 Bilder an und justiere anhand der Daten die Gewichte nach
# weil immer nur sehr wenig justiert wird, muss man die Daten mehrfach durchlaufen
model.fit(
    X_train_mapped,
    y_train,
    epochs=10,
    batch_size=1000
)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8464 - loss: 0.3407
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9264 - loss: 0.1888
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9427 - loss: 0.1628
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9492 - loss: 0.1497
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9505 - loss: 0.1423
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9522 - loss: 0.1353
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9522 - loss: 0.1312
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9535 - loss: 0.1259
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9529 - loss: 0.1258
Epoch 10/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9547 - loss: 0.1216


#### Trainingserfolg prüfen
- Theoretisch kann man mit den vorhandenen Daten, mit denen man trainiert hat, testen.
- Da hier aber etwas (z.B. ein Pixel an einer bestimmten Stelle) erkannt worden sein kann, sollte man sich Daten aufheben, mit denen man nicht lernt und diese später als Testlauf durchlaufen lassen -> Problem "Overfitting"
- Gutes Maß: c.a. 75% Trainingsdaten / 25% Testdaten (bei neuronalen Netzen mit vielen Daten wird auch oft mit 10% Testdaten gearbeitet)

In [14]:
# Test mit vorhandenen, trainierten Daten
print(model.evaluate(X_train_mapped, y_train))
# was sind die zwei ausgegebenen Zahlen?
print(model.metrics_names)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 783us/step - accuracy: 0.9532 - loss: 0.1216
[0.12187030166387558, 0.9535333514213562]
['loss', 'compile_metrics']


In [ ]:
# Test mit Testdaten (-1: zähle selbst die Anzahl an Einträgen)
X_test = open_images("data/fashion/t10k-images-idx3-ubyte.gz").reshape(-1, 784)
y_test = open_labels("data/fashion/t10k-labels-idx1-ubyte.gz") == 0

model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - accuracy: 0.9530 - loss: 0.1249


[0.12708264589309692, 0.9524000287055969]